In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2022-11-13 16:28:45.573758: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 16:28:45.709487: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-13 16:28:46.166421: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-13 16:28:46.166479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
import tensorflow_datasets as tfds

/home/mikheil/.conda/envs/ssl-repo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sys.path.append(str(Path("../../ssl").resolve()))

In [5]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.pi_model.pi_model import PiModelTrainer
from src.trainers.pi_model.pi_model_config import PiModelTrainerConfig
from src.data_loaders.pi_model.pi_model_config import PiModelDataLoaderConfig
from src.data_loaders.pi_model.pi_model import PiModelDataLoader

# Description

In this notebook, the AlexNet model will be trained on the CIFAR-10 dataset using only 25% of the labelled dataset.

## Set up Experiment

In [6]:
class TrainerConfig(PiModelTrainerConfig):
    num_epochs = 200
    loss_ramp_up_epochs = 150
    unsup_loss_weight = 3.0

train_config = TrainerConfig()

In [7]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [8]:
class DataLoaderConfig(PiModelDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 50000 # dataset size    

    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config = DataLoaderConfig()

## Get Datasets

In [9]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [10]:
# get train dataset by subsampling 10% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.10, random_state = 42
)

# cast types
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.int64)
y_train_labelled = y_train_labelled.astype(np.int64)
y_test_full = y_test_full.astype(np.int64)

In [11]:
x_train = np.concatenate((x_train_unlabelled, x_train_labelled), axis = 0)
y_train = np.concatenate((y_train_unlabelled, y_train_labelled), axis = 0)

In [12]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-13 16:28:48.290954: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 16:28:48.298679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 16:28:48.299366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 16:28:48.300530: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [13]:
# create train dataset
train_data = PiModelDataLoader(train_data, data_loader_config)(training = True)

In [14]:
# create test dataset
val_data = PiModelDataLoader(val_data, data_loader_config)(training = False)

In [15]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")

Train dataset size: 782
Validation dataset size: 157


# Train Model

In [16]:
model = PiModel(model_config)()

In [17]:
trainer = PiModelTrainer(
    model, train_data, train_config,
    val_dataset = val_data)

In [18]:
trainer.train()

2022-11-13 16:28:51.608970: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : 2.26. Validation loss is : 2.13. Validation acc. is : 19.75.
Training loss at epoch 1 is : 2.05. Validation loss is : 1.99. Validation acc. is : 26.60.
Training loss at epoch 2 is : 1.96. Validation loss is : 1.86. Validation acc. is : 29.38.
Training loss at epoch 3 is : 1.90. Validation loss is : 1.82. Validation acc. is : 33.21.
Training loss at epoch 4 is : 1.86. Validation loss is : 1.74. Validation acc. is : 34.26.
Training loss at epoch 5 is : 1.81. Validation loss is : 1.67. Validation acc. is : 36.56.
Training loss at epoch 6 is : 1.77. Validation loss is : 1.64. Validation acc. is : 39.37.
Training loss at epoch 7 is : 1.74. Validation loss is : 1.84. Validation acc. is : 35.09.
Training loss at epoch 8 is : 1.69. Validation loss is : 1.60. Validation acc. is : 40.71.
Training loss at epoch 9 is : 1.67. Validation loss is : 1.51. Validation acc. is : 44.84.
Training loss at epoch 10 is : 1.65. Validation loss is : 1.54. Validation acc. is : 42.67